In [1]:
! pip install git+https://github.com/uiuc-focal-lab/syncode.git

  Cloning https://github.com/uiuc-focal-lab/syncode.git to /tmp/pip-req-build-cn9hfuup
  Running command git clone --filter=blob:none --quiet https://github.com/uiuc-focal-lab/syncode.git /tmp/pip-req-build-cn9hfuup
  Resolved https://github.com/uiuc-focal-lab/syncode.git to commit 5ec58cd2a34effa4196c6ed13e73f0480bbf45f3
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/shubhamugare/mxeval.git to /tmp/pip-install-xngztswg/mxeval_bd08680e336d49f3886ac27bb812ad1f
  Running command git clone --filter=blob:none --quiet https://github.com/shubhamugare/mxeval.git /tmp/pip-install-xngztswg/mxeval_bd08680e336d49f3886ac27bb812ad1f
  Resolved https://github.com/shubhamugare/mxeval.git to commit 590129d063c5e7d145e8b15a83f00ff20d69a189
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 78.0 kB/s eta 0:00:00
 

In [2]:
grammar = """
        start: float op float
        float: DEC_NUMBER "." DEC_NUMBER
        op: "+" | "-" | "*" | "/"
        DEC_NUMBER: /0|[1-9]\d*/i
"""

In [3]:
from syncode import Syncode

In [4]:
model_name = "microsoft/phi-2"

# Load the unconstrained original model
llm = Syncode(model = model_name, mode='original', max_new_tokens=20)

# Load the Syncode augmented model
syn_llm = Syncode(model = model_name, mode='grammar_mask', grammar=grammar)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Creating DFA mask store for CodeGenTokenizerFast and custom, may take more than 10 minutes. Caching at /content/cache/mask_stores/CodeGenTokenizerFast/grammar_mask_7842161597_50257.pkl.
Ignore whitespace tokens is False


100%|██████████| 14/14 [00:18<00:00,  1.31s/it]


In [5]:
prompt_template="You will be asked with natural language about floating point arithmetics. Your task is to convert into an expression.\n"
prompt_template += "Example: \nQuestion: What is 45.1 plus 23.54?\nAnswer: 45.1+23.54\n"
prompt_template += "Question: What is 120.4 divided by 4.0?\nAnswer: 120.4/4.0\nQuestion: "

In [7]:
output = llm.infer("What is 2 plus 2?")
print(f"LLM output:\n{repr(output)}\n")

output = syn_llm.infer(prompt_template + "What is 2 plus 2? Answer: ")
print(f"Syncode augmented LLM output:\n{output}")
type(output)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


LLM output:
['\nAnswer: 4\n\nExercise 2:\nWhat is the sum of 5 and 7?']

Syncode augmented LLM output:
['2.0+2.0']


list

In [8]:
def run_syncode(textual_query):
  output = syn_llm.infer(prompt_template + textual_query + " Answer: ")[0]
  splitter = ''
  if output.find('+') != -1:
    splitter = '+'
  if output.find('-') != -1:
    splitter = '-'
  if output.find('*') != -1:
    splitter = '*'
  if output.find('/') != -1:
    splitter = '/'

  final_answer = eval(output)
  output_splitted = output.split(splitter)
  operands = [float(output_splitted[0]), float(output_splitted[1])]
  return operands, splitter, final_answer


In [9]:
run_syncode("The Python program can then calculate 327.0 * 11.0")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


([327.0, 11.0], '*', 3597.0)